<a href="https://colab.research.google.com/github/brs1977/BERT-Transformer-for-Summarization/blob/master/AbsBertSumPredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [1]:
%cd /content
!git clone https://github.com/brs1977/BERT-Transformer-for-Summarization.git bertsum
%cd /content/bertsum

class ARGS(object):
    bert_model = 'bert-base-uncased'
    model_path =  'https://drive.google.com/uc?id=1-AhwiqfWWSLwJuvUo7XpR8vfMd-g9Tw9&export=download' #'https://drive.google.com/uc?id=1-QRy87UMkoPCPVQrNbroC-XrIo1Qmj--&export=download'
    # config_path = 'https://drive.google.com/uc?id=1-0QeYVALlC6iOlN8XEU6SyI6DUd50O5a&export=download'
    # batch_size = 16
    max_src_len = 130

args = ARGS()    

#download model and config
!gdown $args.model_path
# !gdown $args.config_path


/content
Cloning into 'bertsum'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 131 (delta 61), reused 80 (delta 28), pack-reused 0
Receiving objects: 100% (131/131), 148.42 KiB | 868.00 KiB/s, done.
Resolving deltas: 100% (61/61), done.
/content/bertsum
Downloading...
From: https://drive.google.com/uc?id=1-AhwiqfWWSLwJuvUo7XpR8vfMd-g9Tw9
To: /content/bertsum/BertAbsSum_3.bin
816MB [00:11, 70.8MB/s]


#Model

In [2]:
import torch
from model import BertAbsSum
import json
from preprocess import convert_examples_to_features
from pytorch_pretrained_bert.tokenization import BertTokenizer


#create tokenizer
tokenizer = BertTokenizer.from_pretrained(args.bert_model)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#load config
with open('/content/bertsum/config.json', 'r') as f:
    config = json.load(f)

#create model
model = BertAbsSum(args.bert_model, config['decoder_config'], device)
model.load_state_dict(torch.load('/content/bertsum/BertAbsSum_3.bin'))
model.to(device)


def convert_one_example(text, src_max_seq_length, tokenizer):
    src_tokens = tokenizer.tokenize(text)
    if len(src_tokens) > src_max_seq_length - 2:
        src_tokens = src_tokens[:(src_max_seq_length - 2)]
    src_tokens = ["[CLS]"] + src_tokens + ["[SEP]"]

    src_ids = tokenizer.convert_tokens_to_ids(src_tokens)

    src_mask = [1] * len(src_ids)
    src_padding = [0] * (src_max_seq_length - len(src_ids))
    src_ids += src_padding
    src_mask += src_padding

    return torch.tensor([src_ids]), torch.tensor([src_mask])    

def predict(predict_text, beam_size=5, n_best=5):
  """predict function"""
  src, src_mask = convert_one_example(predict_text, args.max_src_len, tokenizer)
  pred, _ = model.beam_decode(src.to(device), src_mask.to(device), beam_size=beam_size, n_best=n_best)  

  # De-tokenize.
  tok_text = " ".join(tokenizer.convert_ids_to_tokens(pred[0][0])).split('[SEP]')[0]
  tok_text = tok_text.replace(" ##", "")
  return tok_text.replace("##", "")  

02/11/2020 08:27:46 - INFO - pytorch_pretrained_bert.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to /tmp/tmpa3brnu7b
100%|██████████| 231508/231508 [00:00<00:00, 932242.95B/s]
02/11/2020 08:27:46 - INFO - pytorch_pretrained_bert.file_utils -   copying /tmp/tmpa3brnu7b to cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
02/11/2020 08:27:46 - INFO - pytorch_pretrained_bert.file_utils -   creating metadata file for /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
02/11/2020 08:27:46 - INFO - pytorch_pretrained_bert.file_utils -   removing temp file /tmp/tmpa3brnu7b
02/11/2020 08:27:46 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.

#Predict

In [3]:

#predict
predict_text = '''Bidirectional Encoder Representations from Transformers (BERT) represents the latest incarnation of pretrained language models which have recently advanced a wide range of natural language processing tasks. In this paper, we showcase how BERT can be usefully applied in text summarization and propose a general framework for both extractive and abstractive models. We introduce a novel document-level encoder based on BERT which is able to express the semantics of a document and obtain representations for its sentences. Our extractive model is built on top of this encoder by stacking several inter-sentence Transformer layers. For abstractive summarization, we propose a new fine-tuning schedule which adopts different optimizers for the encoder and the decoder as a means of alleviating the mismatch between the two (the former is pretrained while the latter is not). We also demonstrate that a two-staged fine-tuning approach can further boost the quality of the generated summaries. Experiments on three datasets show that our model achieves state-of-the-art results across the board in both extractive and abstractive settings.'''
predict(predict_text)


'tensor - level encoderal encoder for text summarization '

In [4]:
predict_text = "we consider the problem of utility maximization for investors with power utility functions. building on the earlier work larsen et al. (2016), we prove that the value of the problem is a frechet-differentiable function of the drift of the price process, provided that this drift lies in a suitable banach space.   we then study optimal investment problems with non-markovian driving processes. in such models there is no hope to get a formula for the achievable maximal utility. applying results of the first part of the paper we provide first order expansions for certain problems involving fractional brownian motion either in the drift or in the volatility. we also point out how asymptotic results can be derived for models with strong mean reversion."
print(predict(predict_text))
'on optimal investment with processes of long or negative memory'


on optimal investment with fractional levy processes 


'on optimal investment with processes of long or negative memory'

In [6]:
predict_text = "in this paper we provide an explicit formula for calculating the boolean number of a ferrers graph. by previous work of the last two authors, this determines the homotopy type of the boolean complex of the graph. specializing to staircase shapes, we show that the boolean numbers of the associated ferrers graphs are the genocchi numbers of the second kind, and obtain a relation between the legendre-stirling numbers and the genocchi numbers of the second kind. in another application, we compute the boolean number of a complete bipartite graph, corresponding to a rectangular ferrers shape, which is expressed in terms of the stirling numbers of the second kind. finally, we analyze the complexity of calculating the boolean number of a ferrers graph using these results and show that it is a significant improvement over calculating by edge recursion."
print(predict(predict_text))
'boolean complexes for ferrers graphs'


boolean complexes for ferrers graphs 


'boolean complexes for ferrers graphs'

In [5]:
predict_text = "kinesin-5, also known as eg5 in vertebrates is a processive motor with 4 heads, which moves on filamentous tracks called microtubules. the basic function of kinesin-5 is to slide apart two anti-parallel microtubules by simultaneously walking on both the microtubules. we develop an analytical expression for the steady-state relative velocity of this sliding in terms of the rates of attachments and detachments of motor heads with the atpase sites on the microtubules. we first analyse the motion of one pair of motor heads on one microtubule and then couple it to the motion of the other pair of motor heads of the same motor on the second microtubule to get the relative velocity of sliding."
print(predict(predict_text))
'relative velocity of sliding of microtubules by the action of kinesin-5'


relative velocity of sliding of the sliding of kinesin - 5 


'relative velocity of sliding of microtubules by the action of kinesin-5'